## This note book is main nate book with will load the Sales data into the postgresql db

In [14]:
import pandas as pd
from datetime import datetime
from pprint import pprint
from AIQSales.Gettoken import AIQ_GetAuth 
from AIQSales.GetData import AIQ_GetData
from AIQSales.LoadToDB import To_postgres

# Get APi access Token
token =  AIQ_GetAuth()
access_token=  token.access_token

# create and object to get data
Data =  AIQ_GetData(access_token)

In [15]:
# Load Sales_df 
Saleslocation=  '../DataFiles/AIQSalesData.csv'
Sales_df =  Data.get_sales(Saleslocation)

In [16]:
# Load Customer_df 
Customerlocation=  'https://jsonplaceholder.typicode.com/users'
Customer_df =  Data.get_Customers(Customerlocation)

In [17]:
location_df =  Customer_df[['id', 'lat', 'lng']]
locationlist = list(location_df.values)

In [19]:
# call weather data funcation to extract the weather information bases on latitude and longitute 
Weather_df = Data.get_WeatherData(locationlist)

[1 '-37.3159' '81.1496']
[2 '-43.9509' '-34.4618']
[3 '-68.6102' '-47.0653']
[4 '29.4572' '-164.2990']
[5 '-31.8129' '62.5342']
[6 '-71.4197' '71.7478']
[7 '24.8918' '21.8984']
[8 '-14.3990' '-120.7677']
[9 '24.6463' '-168.8889']
[10 '-38.2386' '57.2232']


In [20]:
Weather_df.head(1)

,id,lat,lng,weather,description,temp,pressure,humidity,grnd_level,sea_level,visibility,wind_speed,wind_deg,wind_gust,inserted_date,inserted_by,updated_date,updated_by
0,1,-37.3159,81.1496,Rain,light rain,293.08,1021,81,1021,1021,,4.19,303,5.47,2024-03-23 18:27:06.611857,Dharm,2024-03-23 18:27:06.611857,dharm


In [21]:
# convert the datatype to make join
Weather_df = Weather_df.astype({'id': str })
Sales_df = Sales_df.astype({'customer_id': str })

In [22]:
# merge weather and Sales df
SalesData_full =  pd.merge(Sales_df, Weather_df, left_on=['customer_id'], right_on= ['id'], how='inner')

In [23]:
# verify sample Data
SalesData_full.head(1)


,order_id,customer_id,product_id,quantity,price,order_date,id,lat,lng,weather,...,grnd_level,sea_level,visibility,wind_speed,wind_deg,wind_gust,inserted_date,inserted_by,updated_date,updated_by
0,2334,5,40,3,35.6,2022-06-21,5,-31.8129,62.5342,Rain,...,1026,1026,,12.91,142,13.39,2024-03-23 18:27:11.663738,Dharm,2024-03-23 18:27:11.663738,dharm


In [24]:
# load final SalesData_full dataframe into postgresql DB
pg_local= {
    "host"      : "localhost",
    "database"  : "AIQ",
    "user"      : "postgres",
    "password"  : "root"
}

#create a connection to Databases
conn = To_postgres(pg_local)

stgTable='sales_stg'
Table='sales'
keycol= 'order_id'

#trigger upsert method
#conn.UpsertData(SalesData_full, stgTable, Table, keycol)

#trigger Delete-insert method
#conn.deleteInsert(SalesData_full, stgTable, Table, keycol)

conn.deleteInsert(SalesData_full, stgTable, Table, keycol)

Connecting to the PostgreSQL database...
Connection successful
Data laod to Stagging table is done
INSERT INTO sales(order_id,customer_id,product_id,quantity,price,order_date,id,lat,lng,weather,description,temp,pressure,humidity,grnd_level,sea_level,visibility,wind_speed,wind_deg,wind_gust,inserted_date,inserted_by,updated_date,updated_by) SELECT * FROM sales_stg 


'data writting have been completed with delete & insert Method'